In [72]:
%pip install openai graphrag pandas requests python-dotenv langchain numpy tiktoken matplotlib scikit-learn pyyaml pydantic instructor
from IPython.display import clear_output ; clear_output()

In [61]:
from dotenv import load_dotenv
import os
load_dotenv()

is_azure = (
  os.getenv("AZURE_OPENAI_ENDPOINT", default="") != "" and
  os.getenv("OPENAI_API_KEY", default="") == ""
)

GPT_4_O_MODEL_NAME = os.getenv("GPT_4_O_MODEL_NAME", default="gpt-4o")
TEXT_EMBEDDING_3_LARGE_MODEL_NAME = os.getenv("TEXT_EMBEDDING_3_LARGE_MODEL_NAME", default="text-embedding-3-large")

if is_azure:
  AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
  AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
  AZURE_OPENAI_API_VERSION = "2024-05-01-preview"
  from openai import AzureOpenAI
  oai = AzureOpenAI(azure_endpoint=AZURE_OPENAI_ENDPOINT, api_key=AZURE_OPENAI_API_KEY, api_version=AZURE_OPENAI_API_VERSION)
else:
  OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
  from openai import OpenAI
  oai = OpenAI(api_key=OPENAI_API_KEY)

In [62]:
import requests
import os

if not os.path.exists('data'): os.makedirs('data')

if not os.path.exists('data/french_revolution.md'):
  french_revolution = requests.get("https://r.jina.ai/https://en.wikipedia.org/wiki/French_Revolution").text.split('\nSee also')[0]
  with open('data/french_revolution.md', 'w') as f:
    f.write(french_revolution)
else:
  with open('data/french_revolution.md', 'r') as f:
    french_revolution = f.read()

print(french_revolution[:123])

Title: French Revolution

URL Source: https://en.wikipedia.org/wiki/French_Revolution

Published Time: 2001-10-18T00:19:10Z


In [63]:
from langchain.text_splitter import MarkdownTextSplitter
import pandas as pd

if not os.path.exists('data/embeddings.parquet'):
  embeddings = pd.DataFrame(columns=['Topic', 'Text', 'Embedding'])

  splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=250)

  chunks = splitter.split_text(french_revolution)
  chunk_embeddings = oai.embeddings.create(
    input=chunks,
    model=TEXT_EMBEDDING_3_LARGE_MODEL_NAME
  )
  for i, chunk in enumerate(chunks):
    try:
      topic = oai.chat.completions.create(
        model=GPT_4_O_MODEL_NAME,
        messages=[
          {
            "role": "system",
            "content": ("Read the user-provided text carefully and output its topic as a short sentence. "
                        "For example: 'Key events in the life of George Washington', 'Inflation in the Weimar republic'. "
                        "Do not add any additional text such as punctuation, markup, or quoting. Only output the topic.")},
          {"role": "user", "content": chunk}
        ],
        max_tokens=23,
        temperature=0.5,
      ).choices[0].message.content
    except Exception:
      pass
    embeddings.loc[len(embeddings)] = [topic or chunk[:23], chunk, chunk_embeddings.data[i].embedding]
  embeddings.to_parquet('data/embeddings.parquet')
else:
  embeddings = pd.read_parquet('data/embeddings.parquet')

embeddings

,Topic,Text,Embedding
0,French Revolution,Title: French Revolution\n\nURL Source: https:...,"[-0.02485630474984646, 0.0026923343539237976, ..."
1,Key events of the French Revolution,"The Storming of the Bastille, 14 July 1789\n\n...","[-0.005255864467471838, 0.0037901357281953096,..."
2,The French Revolution and its impact on modern...,The French Revolution[a] was a period of polit...,"[-0.027185581624507904, -0.016875529661774635,..."
3,Causes and early events of the French Revolution,Its causes are generally agreed to be a combin...,"[-0.020989766344428062, -0.024809090420603752,..."
4,Key events in the early stages of the French R...,which was converted into a National Assembly i...,"[-0.009496822953224182, -0.017249926924705505,..."
...,...,...,...
296,Alphonse Aulard's contributions to the study o...,evidence.[266][267] Alphonse Aulard (1849–1928...,"[-0.0005632034735754132, 0.021342121064662933,..."
297,Marxist socio-economic analysis of the French ...,Socio-economic analysis and a focus on the exp...,"[-0.003857595380395651, 0.007019456941634417, ..."
298,Alfred Cobban's critique of Jacobin-Marxist in...,Alfred Cobban challenged Jacobin-Marxist socia...,"[0.004117575474083424, 0.00744161382317543, -0..."
299,Political decisions and radicalization during ...,"In their 1965 work, La Revolution française, F...","[0.017132386565208435, 0.012499384582042694, -..."


In [64]:
import numpy as np
import tiktoken

def cosine_similarity(vector1, vector2):
  dot_product = np.dot(vector1, vector2)
  norm1 = np.linalg.norm(vector1)
  norm2 = np.linalg.norm(vector2)
  similarity = dot_product / (norm1 * norm2)
  return similarity

tokenizer = tiktoken.encoding_for_model('gpt-4o')

def embeddings_search(query, max_tokens=10000, k=100, min_similarity=0.2):
  query_embedding = oai.embeddings.create(
    input=[query],
    model=TEXT_EMBEDDING_3_LARGE_MODEL_NAME
  ).data[0].embedding
  results = embeddings.copy()
  results['Similarity'] = results['Embedding'].apply(lambda x: cosine_similarity(x, query_embedding))
  results = results.sort_values(by='Similarity', ascending=False).head(k)
  results = results[results['Similarity'] >= min_similarity]
  results['Tokens'] = results['Text'].apply(lambda txt: len(tokenizer.encode(txt)))
  while results['Tokens'].sum() > max_tokens:
    results = results[:-1]
  return results

In [73]:
DEFAULT_RESPONSE_TYPE = 'Summarize and explain in 1-2 paragraphs with bullet points using at most 300 tokens'
DEFAULT_MAX_CONTEXT_TOKENS = 10000

from graphrag.query.structured_search.global_search.reduce_system_prompt import REDUCE_SYSTEM_PROMPT

def ask_embeddings(query, response_type=DEFAULT_RESPONSE_TYPE):
  results = embeddings_search(query, max_tokens=DEFAULT_MAX_CONTEXT_TOKENS)
  response = oai.chat.completions.create(
    model=GPT_4_O_MODEL_NAME,
    messages=[
      {
        "role": "system",
        "content": REDUCE_SYSTEM_PROMPT.format(
          response_type=response_type,
          report_data="---\n---\n".join(results['Text'].tolist()),
        ),
      },
      {"role": "user", "content": query}
    ],
    max_tokens=4000,
    temperature=0.5,
  ).choices[0].message.content
  return response

In [74]:
from IPython.display import Markdown

result = ask_embeddings('Who was Robespierre and what was his role in the French revolution?')

Markdown(result)

### Maximilien Robespierre's Role in the French Revolution

Maximilien Robespierre was a prominent and controversial figure during the French Revolution, known for his radical stances and leadership during the Reign of Terror. His actions and policies significantly influenced the course of the Revolution.

#### Key Points

- **Political Leadership**: Robespierre was a leading member of the radical Montagnard faction and played a crucial role in the Committee of Public Safety, which became the de facto executive government during the Reign of Terror [Data: Reports (7, 111, 114, +more)].
- **Reign of Terror**: Under his influence, the Law of 22 Prairial was enacted, which expedited the execution of "enemies of the people" and led to a significant increase in executions in Paris [Data: Reports (129, 132, 133, +more)]. Approximately 16,000 people were executed, and another 40,000 may have been summarily executed or died awaiting trial during this period [Data: Reports (118, 119)].
- **Radical Reforms**: Robespierre supported radical reforms, including universal male suffrage and the suspension of the 1793 Constitution to maintain revolutionary control [Data: Reports (114, 130, +more)].
- **Downfall and Execution**: His refusal to name alleged conspirators against the Republic led to his downfall. After a failed suicide attempt, Robespierre was arrested and executed on 28 July 1794, marking the end of the Reign of Terror [Data: Reports (131, 132, 133)].
- **Legacy**: Despite his execution, some of his allies, like Paul Barras and Joseph Fouché, retained their positions and continued to influence French politics [Data: Reports (136, 137)].

Robespierre's tenure was marked by his unwavering commitment to revolutionary ideals and his willingness to use extreme measures to achieve them. His leadership during the Reign of Terror remains one of the most debated aspects of the French Revolution.

In [75]:
result = ask_embeddings('Timeline of the French revolution')

Markdown(result)

### Timeline of the French Revolution

The French Revolution was a period of profound political and societal change in France from 1789 to 1799. Below are key events and milestones:

- **May 1789**: The Estates General convened, marking the beginning of the Revolution [Data: Reports (3, 4, 5, 6, +more)].
- **14 July 1789**: Storming of the Bastille, a significant event symbolizing the uprising against the monarchy [Data: Reports (1, 6, 48, 49, +more)].
- **August 1789**: The National Assembly abolished feudalism and declared the rights of man [Data: Reports (56, 57, 58, 61, +more)].
- **September 1792**: Proclamation of the French First Republic following the insurrection and abolition of the monarchy [Data: Reports (100, 101, 106, 109, +more)].
- **January 1793**: Execution of Louis XVI, escalating the revolutionary fervor and leading to increased conflict [Data: Reports (108, 109, 110, 114, +more)].
- **1793-1794**: Reign of Terror, marked by mass executions and political purges [Data: Reports (118, 119, 121, 122, +more)].
- **July 1794**: Fall of Robespierre, ending the Reign of Terror [Data: Reports (134, 135, 158, +more)].
- **1795**: Establishment of the Directory, a new government structure [Data: Reports (111, 116, 121, +more)].
- **November 1799**: Coup of 18 Brumaire, leading to the rise of Napoleon Bonaparte and the end of the Revolution [Data: Reports (159, 160, 161, +more)].

### Key Outcomes

- **Abolition of the Ancien Régime**: The Revolution dismantled the feudal system and abolished the privileges of the nobility [Data: Reports (55, 56, 234, +more)].
- **Creation of the French First Republic**: The monarchy was replaced by a republic, introducing new political and social reforms [Data: Reports (106, 109, 110, +more)].
- **Reign of Terror**: A period of extreme violence aimed at purging counter-revolutionaries [Data: Reports (118, 119, 121, 122, +more)].
- **Rise of Napoleon**: The Revolution ended with Napoleon's coup, setting the stage for his subsequent rule [Data: Reports (158, 159, 160, +more)].

These events collectively transformed France, influencing modern political and social structures worldwide.

In [76]:
import yaml

if not os.path.exists('data/graphrag'):
  !python -m graphrag.index --init --root data/graphrag

with open('data/graphrag/settings.yaml', 'r') as f:
  settings_yaml = yaml.load(f, Loader=yaml.FullLoader)
settings_yaml['llm']['model'] = GPT_4_O_MODEL_NAME
settings_yaml['llm']['api_key'] = AZURE_OPENAI_API_KEY if is_azure else OPENAI_API_KEY
settings_yaml['llm']['type'] = 'azure_openai_chat' if is_azure else 'openai_chat'
settings_yaml['embeddings']['llm']['api_key'] = AZURE_OPENAI_API_KEY if is_azure else OPENAI_API_KEY
settings_yaml['embeddings']['llm']['type'] = 'azure_openai_embedding' if is_azure else 'openai_embedding'
settings_yaml['embeddings']['llm']['model'] = TEXT_EMBEDDING_3_LARGE_MODEL_NAME
if is_azure:
  settings_yaml['llm']['api_version'] = AZURE_OPENAI_API_VERSION
  settings_yaml['llm']['deployment_name'] = GPT_4_O_MODEL_NAME
  settings_yaml['llm']['api_base'] = AZURE_OPENAI_ENDPOINT
  settings_yaml['embeddings']['llm']['api_version'] = AZURE_OPENAI_API_VERSION
  settings_yaml['embeddings']['llm']['deployment_name'] = TEXT_EMBEDDING_3_LARGE_MODEL_NAME
  settings_yaml['embeddings']['llm']['api_base'] = AZURE_OPENAI_ENDPOINT

with open('data/graphrag/settings.yaml', 'w') as f:
  yaml.dump(settings_yaml, f)

if not os.path.exists('data/graphrag/input'):
  os.makedirs('data/graphrag/input')
  !cp data/french_revolution.md data/graphrag/input/french_revolution.txt
  !python -m graphrag.index --root ./data/graphrag

In [77]:
import subprocess

def ask_graph(query, response_type=DEFAULT_RESPONSE_TYPE):
  env = os.environ.copy() | {
    'GRAPHRAG_GLOBAL_SEARCH_MAX_TOKENS': str(DEFAULT_MAX_CONTEXT_TOKENS),
  }
  command = [
    'python', '-m', 'graphrag.query',
    '--root', './data/graphrag',
    '--method', 'local',
    '--response_type', response_type,
    query,
  ]
  output = subprocess.check_output(command, universal_newlines=True, env=env)
  return output.split('Search Response: ')[1]

In [78]:
from IPython.display import Markdown

result = ask_graph('Who was Robespierre and what was his role in the French revolution?')

Markdown(result)

# Maximilien Robespierre and His Role in the French Revolution

## Overview
Maximilien Robespierre was a central figure in the French Revolution, known for his radical views and significant influence within the revolutionary government. His actions and policies were pivotal during the Reign of Terror, a period marked by extreme political repression and mass executions.

## Key Roles and Actions
- **Leadership and Influence**:
  - Robespierre was a leading member of the Committee of Public Safety and a prominent leader within the Jacobins and the Montagnard faction [Data: Entities (52); Relationships (136, 263)].
  - He opposed property qualifications for voting and advocated for broader democratic participation [Data: Entities (52, 133); Relationships (269, 280)].

- **Reign of Terror**:
  - Robespierre's policies were central to the Reign of Terror, which saw mass executions of perceived enemies of the revolution, including notable figures like Georges Danton and Hébert [Data: Entities (52, 186); Relationships (97, 266, 267)].
  - His radical stance and actions eventually led to his downfall, culminating in his arrest and execution on 28 July 1794 [Data: Entities (52); Relationships (275)].

- **Cult of the Supreme Being**:
  - Robespierre led the establishment of the Cult of the Supreme Being, a revolutionary cult that faced ridicule and opposition [Data: Entities (249); Relationships (268)].

- **Political Conflicts**:
  - He was involved in significant political conflicts, including opposition to the refractory clergy and the Convention, which eventually authorized his arrest [Data: Relationships (261, 275)].

Robespierre's legacy is marked by his intense ideological commitment and the extreme measures he endorsed to achieve his revolutionary goals. His influence and actions left a lasting impact on the course of the French Revolution.


In [79]:
result = ask_graph('Timeline of the French revolution')

Markdown(result)

# Timeline of the French Revolution

The French Revolution was a decade-long period of significant political and social upheaval in France, marked by key events and transformations:

- **1789**: The revolution began with the Estates General convening in May, followed by the Storming of the Bastille on July 14, symbolizing the uprising against the monarchy [Data: Relationships (0, 9, 59)].
- **1791**: The Constitution of 1791 established a constitutional monarchy, but political struggles continued [Data: Entities (3, 80)].
- **1792**: The monarchy was abolished, and the French First Republic was proclaimed in September [Data: Entities (5); Relationships (4)].
- **1793**: King Louis XVI was executed in January, marking the start of the Reign of Terror, a period of extreme political repression and mass executions [Data: Entities (3, 6); Relationships (2, 5)].
- **1794**: The Reign of Terror ended with the execution of Robespierre in July [Data: Entities (6)].
- **1799**: The revolution concluded with the coup of 18 Brumaire in November, leading to the establishment of the French Consulate and the rise of Napoleon Bonaparte [Data: Entities (0, 7); Relationships (34, 178)].

These events collectively reshaped France's political landscape, leading to the end of the Ancien Régime and the rise of republicanism.
